In [1]:
import os

In [2]:
os.chdir("../")

In [12]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PartialModelTrainerConfig:
    root_dir: Path
    X_train_data_path: Path
    y_train_data_path: Path
    partial_model_name: str

In [16]:
from src.StockSeer.constants import *
from src.StockSeer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_partial_model_trainer_config(self) -> PartialModelTrainerConfig:
        config = self.config.partial_model_trainer

        create_directories([config.root_dir])

        partial_model_trainer_config = PartialModelTrainerConfig(
            root_dir=config.root_dir,
            X_train_data_path = config.X_train_data_path,
            y_train_data_path = config.y_train_data_path,
            partial_model_name = config.partial_model_name
        )

        return partial_model_trainer_config

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import ReduceLROnPlateau
import tensorflow as tf
from src.StockSeer.logging import logger
import numpy as np
class PartialModelTrainer:
    def __init__(self, config: PartialModelTrainerConfig):
        self.config = config

    
    def build_lstm_model(self,input_shape, lstm_units=[50, 50, 50], dense_units=1, optimizer='adam', loss='mean_squared_error'):
        model = Sequential()

        for i, units in enumerate(lstm_units):
            if i == 0:
                model.add(LSTM(units=units, return_sequences=True, input_shape=input_shape))
            else:
                model.add(LSTM(units=units, return_sequences=(i < len(lstm_units)-1)))

        model.add(Dense(units=dense_units))

        model.compile(optimizer=optimizer, loss=loss)
        logger.info("model built succesfully")
        return model




    def partial_train(self):

        np.random.seed(42)
        tf.random.set_seed(42)
        # Assuming x_train.shape[1] is the time steps and 1 is the feature dimension
        X_train = np.load(self.config.X_train_data_path)
        y_train = np.load(self.config.y_train_data_path)
    
        input_shape = (X_train.shape[1], 1)
     
        # Define LSTM model with specified architecture
        lstm_model = self.build_lstm_model(input_shape=input_shape)
        lr_callback = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=2) 
        # Train the model
        logger.info("partial model training started")
        lstm_model.fit(X_train, y_train, epochs=1, batch_size=32, callbacks=[lr_callback])
        lstm_model.save(self.config.partial_model_name)
        logger.info(f"partial model trainned and save at {self.config.partial_model_name}")

In [34]:
try:
    config = ConfigurationManager()
    partial_model_trainer_config = config.get_partial_model_trainer_config()
    partial_model_trainer_config = PartialModelTrainer(partial_model_trainer_config)
    partial_model_trainer_config.partial_train()
except Exception as e:
    raise e


[2024-03-05 09:04:59,388: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-05 09:04:59,388: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-05 09:04:59,393: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-05 09:04:59,395: INFO: common: created directory at: artifacts]
[2024-03-05 09:04:59,397: INFO: common: created directory at: artifacts/partial_model_trainer]
(10595, 100, 1)
(100, 1)
(100, 1)
[2024-03-05 09:05:00,426: INFO: 1086933789: model built succesfully]
[2024-03-05 09:05:00,428: INFO: 1086933789: partial model training started]
332/332 [==============================] - 35s 86ms/step - loss: 0.0269 - lr: 0.0010
[2024-03-05 09:05:36,029: INFO: 1086933789: partial model trainned and save at partial_model.h5]


c:\Users\Krish Goyani\Desktop\DS Projects\Stock-Seer\env\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
